# IYP Query API - Essential Features Test (Fixed)

Quick demonstration of core capabilities with automatic alias generation

In [ ]:
# Setup - Import and Connect
from iyp_query import connect, Q, And, Or
import pandas as pd

# Connect to database
iyp = connect('bolt+s://iyp.christyquinn.com:7687', 'neo4j', 'lewagon25omgbbq')
print("✅ Connected to IYP database")

## 1. Basic Query - Find AS by Number (Automatic Alias)

In [ ]:
# Find a specific AS (Google) - using automatic alias generation
result = (iyp.builder()
          .find('AS', asn=15169)  # Will automatically use 'as' as alias
          .return_fields(['as.asn', 'as.name'])
          .run())

print(f"AS15169: {result[0]['as.name'] if result else 'Not found'}")

## 2. Relationship Traversal - Find Upstream Providers

In [ ]:
# Find upstream providers using high-level API
providers = iyp.find_upstream_providers(asn=216139, max_hops=1)

print("Upstream providers for AS216139:")
for p in providers[:3]:  # Show first 3
    print(f"  - AS{p.get('upstream.asn', 'N/A')}: {p.get('upstream.name', 'Unknown')}")

## 3. Complex Conditions - Boolean Logic (Automatic Aliases)

In [ ]:
# Find US-based ASes that depend on major transit providers - with automatic aliases
major_transit = [174, 3356, 1299]  # Cogent, Level3, Telia

results = (iyp.builder()
           .find('AS')  # Will use 'as' alias automatically
           .with_relationship('DEPENDS_ON', to='AS')  # Will use 'as_1' for the provider AS
           .with_relationship('COUNTRY', to='Country')  # Will use 'country' alias automatically
           .where(
               And(
                   Q('as_1.asn').in_(major_transit),
                   Q('country.country_code') == 'US'
               )
           )
           .limit(3)
           .return_fields(['as.asn', 'as.name', 'as_1.asn'])
           .run())

print(f"Found {len(results)} US customers of major transit providers")
for r in results:
    print(f"  AS{r['as.asn']} -> AS{r['as_1.asn']}")

## 4. Graph Traversal - Multi-hop Relationships

In [ ]:
# Find 2-hop upstream providers - let automatic aliases handle conflicts
results = (iyp.builder()
           .find('AS', asn=216139)  # 'as' alias
           .upstream(hops=2)  # Will create unique alias for upstream nodes
           .limit(5)
           .run())

print("2-hop upstream from AS216139:")
for r in results:
    # Print all fields to see what aliases were generated
    print(f"  Result: {r}")

## 5. Export to DataFrame with Automatic Aliases

In [ ]:
# Get results as pandas DataFrame - let system generate aliases
df = (iyp.builder()
      .find('AS')  # Will use 'as' alias
      .with_relationship('COUNTRY', to='Country')  # Will use 'country' alias
      .where(Q('country.country_code') == 'GB')
      .limit(10)
      .return_fields(['as.asn', 'as.name', 'country.country_code'])
      .run_as_dataframe())

print(f"UK ASes (showing {len(df)} rows):")
print(df[['as.asn', 'as.name']].head() if not df.empty else "No results")

## 6. Debug - View Generated Cypher with Auto-Aliases

In [ ]:
# See what Cypher query is generated with automatic aliases
query = (iyp.builder()
         .find('AS')  # Will use 'as' alias
         .with_relationship('PEERS_WITH', to='AS')  # Will use 'as_1' alias for peer
         .where(Q('as.asn') == 15169)
         .limit(2)
         .return_fields(['as.asn', 'as_1.asn', 'as_1.name']))

print("Generated Cypher with automatic aliases:")
print(query.to_cypher())
print("\nParameters:")
print(query.get_parameters())

## Summary of Fixed Features

✅ **Automatic Alias Generation** - No more alias conflicts!  
✅ **Basic Queries** - Find nodes without worrying about aliases  
✅ **Relationships** - System handles unique aliases for each node  
✅ **Boolean Logic** - Works with auto-generated aliases  
✅ **Multi-hop Traversal** - Automatic unique naming  
✅ **DataFrame Export** - Clean column names from auto-aliases  
✅ **Debug Support** - See exactly what aliases were generated  